# Sentence Classification of Security Questions

This approach examines how to classify whether questions relate to security or if they refer to the installation of doors or windows. 

The approach produces a classifier with good accuracy that takes between 0.1 and 0.2 seconds to execute.

# Install Required Packages

In [ ]:
! pip install xgboost

MTEB is the Massive Text Embeddings Benchmark. 

In [ ]:
! pip install mteb

<b>Datasets</b>

Datasets is a library created by Hugging Face that is useful for easily accessing datasets for NLP tasks. 


Datasets is a requirement for the mteb package. On my system, I just found it was easier to make sure that this package was installed. 

In [ ]:
! pip install datasets

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


## MTEB

1) download MTEB
2) use embeddings with classification


https://github.com/embeddings-benchmark/mteb

In [1]:
from mteb import MTEB

# General

In [2]:
import sys

In [3]:
sys.path.append("/home/scott/whiteowlconsultinggroup/cust/sedw/maincode")
sys.path.append("/home/scott/whiteowlconsultinggroup/cust/sedw/maincode/api")
sys.path.append("/home/scott/whiteowlconsultinggroup/cust/sedw")
sys.path.append("/home/scott/whiteowlconsultinggroup/cust/sedw/maincode/api/wocg/classifiers/security")

In [4]:
import util as ut

In [5]:
import denseencoder as de

## Collect sentences for training and testing

In [6]:
import importlib

In [7]:
import torch

In [8]:
torch.cuda.is_available()

True

In [9]:
import time
import os

In [10]:
window_and_door_questions = [
    "What kind of warranties does SEDW offer on window installation?",
    "What kinds of Flush Doors does Barnett Millworks offer?",
    "I am finishing a door. List out for me the SEDW process for doing this.",
    "I am an employee at SEDW. How long do I have to wait until I get health insurance?",
    "I want to understand more about the performance of Andersen Multiglide doors. Can you put together a markdown table for me that gives a comparison of Traditional Alum-Clad Wood Non-Thermal vs Contemporary Alum-Class Wood Thermally controlled. These categories as columns and I want to see configuration, Sill Type, Design Pressure, and Water Test Pressure as rows?",
    "Put together a markdown table for AAN3218, AA32110, AAN3428, and AAN3830. I want the columns for this table to be clear opening area, glass area, and overall window area.",
    "provide a QAC for remodel / replacement jobs",
    "list the items on our Quality Assurance Checklist",
    "can an employee take what looks like extra product from a job and keep it for personal use?",
    "can i pay for lunch with the company credit card?",
    "Who does What at Sedw?",
    "what are the procedures to fix the zebra label printer",
    "what does anodized mean?",
    "what are the types of door stops offered by emtek?",
    "what is a good neck stop?",
    "explain door handing",
    "is there a document that shows images or examples of door handing",
    "what's the difference in an outswing door and an inswing door",
    "what are muntin bars?",
    "what is a sash?",
    "what is multi-point hardware?",
    "what hardware finishes are available for andersen series 400 windows?",
    "you provided 7 finishes. there should be a total of 12 finishes available.",
    "what is an extension jamb?",
    "When I am checking acknowledgements (for a new construction project) to ensure orders are accurate, what should I check for?",
    "what information / documents should I use to accomplish this?",
    "what type of documents would I use to accomplish this?",
    "what should I check for when reviewing new construction project documents?",
    "What are the most common window frame materials used by Andersen?",
    "How do I properly measure for a replacement window?",
    "What tools do I need for installing a pre-hung exterior door?",
    "What is the warranty on Milgard vinyl windows?",
    "How do I clean vinyl window frames?", 
    "What is the best way to weatherproof doors and windows?",
    "How often should I lubricate and adjust sliding patio doors?",
    "What is the proper installation method for bay and bow windows?",
    "What is the maximum recommended window screen size for Andersen casement windows?",
    "What is the best way to repair fogged double pane windows?",
    "How do I adjust self-closing door hinges?",
    "What is the proper installation clearance for fire rated doors?",
    "What are signs that a wood door may be warped and need replacement?",
    "How do I fill gaps between window frames and siding?",
    "What are the most energy efficient styles of windows?",
    "How do I remove paint from door hinges?",
    "What are the tools needed for installing a storm door?",
    "What are the benefits of using foam insulation around windows?",
    "How do I fix a sticky door lock?",
    "What maintenance is required for steel entry doors?",
    "What are the signs of failing weather stripping on doors?",
    "How do I remove scratches from a glass shower door?",
    "What is the proper installation method for sliding barn doors?",
    "How do I repair torn window screens?",
    "What are the best practices for painting exterior doors?",
    "How often should patio door rollers be cleaned and lubricated?",
    "What causes condensation between patio door panes?",
    "How do I fix a crooked door that sticks?",
    "What are signs that a door threshold needs to be replaced?",
    "How do I fix a double hung window that is hard to open?",
    "What is the proper procedure for removing old windows?"
]


document_security_questions = [
    "List all of the users in the system.",
    "Use the GCP API to list all of the users in the system.",
    "Add Christina Hoelscher with email christina@yahoo.com to the system.",
    "How to manage user permissions?",
    "Is christina@yahoo.com in the system?",
    "Use the GCP API to add the role installer to the document named a-series-brochure.pdf",
    "Remove christina@yahoo.com from the system.",
    "Use the GCP API to remove ralph.a.brooks@gmail.com from the system",
    "What roles does bob@andersen.com have?",
    "Add the accounting role to jane@fbook.com .",
    "Remove the warehouse role from sally@oregonwindows.com.", 
    "List all of the documents in the system.",
    "List all of the documents in the system that have a filename that starts with warranty.",
    "Add the role clerical to the document named sedw-companypolicies2017-12-27.pdf",
    "Remove the role support from the document named support-instructions.pdf",
    "What is the process for granting access to new documents?",
    "How do I check who has access to a specific document?",  
    "Can you provide a list of users with editor access to the document project-plans.docx?",
    "How do I remove a document from the system?",
    "What user management features does the system provide?",
    "How are user permissions managed?",
    "List all users with viewer access to the document quarterly-financials.xlsx",
    "How do I add a new user to the system?",  
    "What authorization levels are available for users?",
    "How do I change a user's authorization level?",
    "List all documents that user john@company.com has editor access to",
    "How do I revoke a user's access to the system?",
    "What auditing capabilities does the system have for user actions?",
    "Provide a list of users who have downloaded the document acme-prospectus.pdf in the past week",
    "How are documents uploaded into the system?",
    "What metadata is stored for each document?",  
    "List all documents modified by user jane@company.com in the past month",
    "How do I change ownership of a document?",
    "What reports are available on user activity and access?",
    "Provide a list of users who have viewer access to documents containing the keyword 'financial'",
    "How are system permissions managed?",
    "What authentication methods are supported?",
    "List all documents that user bob@company.com has downloaded in the past month",
    "How do I reset a user's password?",
    "What options are available for creating user groups?",
    "Provide a list of users with editor access to documents containing the keyword 'contract'",
    "How do I create a new role in the system?",
    "What default roles exist?",
    "List all documents that the HR role has editor access to", 
    "How do I remove a role from the system?",
    "What user audit logging does the system perform?",
    "Provide a list of roles that have access to the document budget-presentation.pptx",
    "How do I edit role permissions?",
    "What reports are available on role activity?",
    "List all roles that have viewer access to documents owned by user sally@company.com",
    "Can you explain how the system handles authentication?",
    "What protocols are supported for integrating with identity providers?",
    "List all documents accessed by users authenticated via social login in the past month",
    "Provide a list of documents that contain sensitive financial keywords",
    "How do I configure restricted access settings for sensitive documents?",
    "What reporting exists for suspicious access attempts?",
    "List all failed login attempts in the past 24 hours",
    "can you check this document? 400-series-brochure.pdf",
    "give me a summary of the contents in 400-series-brochure.pdf",
    
]

In [11]:
len(window_and_door_questions), len(document_security_questions)

(59, 59)

A leaderboard of text embeddings is hosted by Hugging Face. The leaderboard uses the Massive Text Embedding Benchmark. https://huggingface.co/spaces/mteb/leaderboard

# e5-small-v2

e5 uses contrastive pre-training with weak supervision. 

https://arxiv.org/pdf/2212.03533v1.pdf - Paper that discusses

* e5-small-v2 has 12 layers and has a embedding size of 384. 

https://huggingface.co/intfloat/e5-small-v2

I thought that e5 could deal with 512 tokens to classify.

# e5-small-v2 (0.13 GB)

In [12]:
import torch.nn.functional as F

from torch import Tensor
from transformers import AutoTokenizer, AutoModel



In [13]:
# This is an example of how the tokenizer is created. In the remaining part of the code, this is part of the DenseEncoder class.

tokenizer = AutoTokenizer.from_pretrained('intfloat/e5-small-v2')

In [14]:

# This is an example of how the encoder is created from the pretrained model. 
# In the remaining part of the code, this will be part of the DenseEncoder class.

encoder = AutoModel.from_pretrained('intfloat/e5-small-v2')

## Train model

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, average_precision_score, f1_score
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import random

In [16]:
scores = {}

In [17]:
batch_size = 32

Logistic Regression (LogisticRegression):

Purpose: Used for classification tasks. It estimates the probability that a given instance belongs to a particular category.
Equation: Uses the logistic function to squeeze the output of a linear equation between 0 and 1.

Logisitic regression makes use of the sigmoid function which is:

$$
P(y = 1 | x) = \frac{1}{1 + e^{-(\beta_0 + \beta_1 x)}}
$$

where $b_0$ represents the log odds outcome when all of the predictor variables are 0. 

In [18]:
logistic_clf = LogisticRegression(
    max_iter=100, 
    random_state=42,
    n_jobs=-1,
    verbose=1
)

In [19]:
# We want to predict if we are getting a general question (0) or a question about security (1)
all_questions = window_and_door_questions + document_security_questions
all_labels = [0]*len(window_and_door_questions) + [1]*len(document_security_questions)

In [20]:
all_question_ids = list(range(len(all_questions)))

In [21]:
# We are going to learn on 80% of the examples
train_ids, test_ids = train_test_split(all_question_ids, test_size=0.2, random_state=42)


In [22]:
len(train_ids), len(test_ids)

(94, 24)

In [23]:
sentences_train = [all_questions[i] for i in train_ids]
y_train = [all_labels[i] for i in train_ids]
sentences_test = [all_questions[i] for i in test_ids]
y_test = [all_labels[i] for i in test_ids]

In [24]:
len(sentences_train), len(y_train), len(sentences_test), len(y_test)

(94, 94, 24, 24)

In [25]:
sentences_test[0:10]

['What are signs that a door threshold needs to be replaced?',
 'What metadata is stored for each document?',
 'I want to understand more about the performance of Andersen Multiglide doors. Can you put together a markdown table for me that gives a comparison of Traditional Alum-Clad Wood Non-Thermal vs Contemporary Alum-Class Wood Thermally controlled. These categories as columns and I want to see configuration, Sill Type, Design Pressure, and Water Test Pressure as rows?',
 'What are the benefits of using foam insulation around windows?',
 'what type of documents would I use to accomplish this?',
 'can you check this document? 400-series-brochure.pdf',
 'Remove the warehouse role from sally@oregonwindows.com.',
 'Who does What at Sedw?',
 'List all documents modified by user jane@company.com in the past month',
 'Remove the role support from the document named support-instructions.pdf']

In [26]:
y_test[0:10]

[0, 1, 0, 0, 0, 1, 1, 0, 1, 1]

In [27]:
# Where is the formal definition for model encode

### Logic for encode

In [28]:
importlib.reload(de)

<module 'denseencoder' from '/home/scott/whiteowlconsultinggroup/cust/sedw/maincode/api/wocg/classifiers/security/denseencoder.py'>

#### Brief explanation of DenseEncoder

In [29]:
len(sentences_train)

94

In [30]:
sentences = sentences_train

In [31]:
from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModel, AutoTokenizer, DataCollatorWithPadding, PreTrainedTokenizerFast, BatchEncoding
from typing import List, Dict
from functools import partial

In [32]:
dataset: Dataset = Dataset.from_dict({'input_texts': sentences})

In [33]:
# Hard coded args
prompt = "query: "
pool_type = "avg"
l2_normalize = False

In [34]:
device = "cuda"

In [35]:
# The transform function applies the tokenizer. 

def _transform_func(tokenizer: PreTrainedTokenizerFast,
                    examples: Dict[str, List]) -> BatchEncoding:
    """
    Use the tokenizer associated with the model to encode the sentences.
    """
    if prompt:
        examples['input_texts'] = [prompt + t for t in examples['input_texts']]

    # Take in the sentences. Pad them to the max length of 512. Then truncate them to 512.
    batch_dict = tokenizer(examples['input_texts'],
                           max_length=512,
                           padding=True,
                           truncation=True)

    return batch_dict

In [36]:
model_name_or_path = "intfloat/e5-small-v2"

In [37]:
atokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

In [38]:
# there is a partial completion of the transform function with the tokenizer
dataset.set_transform(partial(_transform_func, atokenizer))

In [39]:
data_collator = DataCollatorWithPadding(atokenizer, pad_to_multiple_of=8)

In [40]:
gpu_count = 1

In [41]:
# 128 on the GPU or one on the CPU
effective_batch_size = max(1, 128 * gpu_count)

In [42]:
data_loader = DataLoader(
    dataset,
    batch_size=effective_batch_size,
    shuffle=False,
    drop_last=False,
    num_workers=2,
    collate_fn=data_collator,
    pin_memory=True)

In [43]:
encoded_embeds = []

In [44]:
batch_dict = next(iter(data_loader))

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [45]:
batch_dict.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [46]:
batch_dict.items()

dict_items([('input_ids', tensor([[  101, 23032,  1024,  ...,     0,     0,     0],
        [  101, 23032,  1024,  ...,     0,     0,     0],
        [  101, 23032,  1024,  ...,     0,     0,     0],
        ...,
        [  101, 23032,  1024,  ...,     0,     0,     0],
        [  101, 23032,  1024,  ...,     0,     0,     0],
        [  101, 23032,  1024,  ...,     0,     0,     0]])), ('token_type_ids', tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])), ('attention_mask', tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]))])

In [47]:
# Take everything in the batch and place it on the GPU
batch_dict = {k: v.to(device) for k, v in batch_dict.items()}

In [48]:
batch_dict

{'input_ids': tensor([[  101, 23032,  1024,  ...,     0,     0,     0],
         [  101, 23032,  1024,  ...,     0,     0,     0],
         [  101, 23032,  1024,  ...,     0,     0,     0],
         ...,
         [  101, 23032,  1024,  ...,     0,     0,     0],
         [  101, 23032,  1024,  ...,     0,     0,     0],
         [  101, 23032,  1024,  ...,     0,     0,     0]], device='cuda:0'),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]], device='cuda:0'),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')}

In [49]:
encoder.to("cuda")

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 384, padding_idx=0)
    (position_embeddings): Embedding(512, 384)
    (token_type_embeddings): Embedding(2, 384)
    (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=384, out_features=384, bias=True)
            (key): Linear(in_features=384, out_features=384, bias=True)
            (value): Linear(in_features=384, out_features=384, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=384, out_features=384, bias=True)
            (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [50]:
from transformers.modeling_outputs import BaseModelOutput

In [51]:
outputs: BaseModelOutput = encoder(**batch_dict)

In [52]:
outputs.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [53]:
outputs

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.3201,  0.1162,  0.3590,  ...,  0.1849, -0.0870,  0.3112],
         [-0.4252,  0.2681,  0.5117,  ...,  0.1928, -0.1562,  0.4421],
         [-0.4626,  0.2505,  0.5180,  ...,  0.2261, -0.1439,  0.5052],
         ...,
         [-0.6241,  0.3687,  0.5437,  ...,  0.3185, -0.2880,  0.4295],
         [-0.6291,  0.3671,  0.5468,  ...,  0.3206, -0.2975,  0.4136],
         [-0.3198,  0.1164,  0.3589,  ...,  0.1845, -0.0870,  0.3114]],

        [[-0.4048,  0.1326,  0.3024,  ...,  0.1842, -0.0558,  0.2753],
         [-0.3959,  0.3549,  0.2270,  ...,  0.2509,  0.1121,  0.4615],
         [-0.4324,  0.3416,  0.2218,  ...,  0.2357,  0.1719,  0.5027],
         ...,
         [-0.5038,  0.4114,  0.3261,  ...,  0.1328,  0.2192,  0.5432],
         [-0.5051,  0.4052,  0.3402,  ...,  0.1590,  0.2244,  0.5482],
         [-0.4927,  0.4033,  0.3281,  ...,  0.1604,  0.1894,  0.5238]],

        [[-0.3173,  0.0634,  0.2716,  ...,  0.1602, -

In [54]:
outputs.last_hidden_state

tensor([[[-0.3201,  0.1162,  0.3590,  ...,  0.1849, -0.0870,  0.3112],
         [-0.4252,  0.2681,  0.5117,  ...,  0.1928, -0.1562,  0.4421],
         [-0.4626,  0.2505,  0.5180,  ...,  0.2261, -0.1439,  0.5052],
         ...,
         [-0.6241,  0.3687,  0.5437,  ...,  0.3185, -0.2880,  0.4295],
         [-0.6291,  0.3671,  0.5468,  ...,  0.3206, -0.2975,  0.4136],
         [-0.3198,  0.1164,  0.3589,  ...,  0.1845, -0.0870,  0.3114]],

        [[-0.4048,  0.1326,  0.3024,  ...,  0.1842, -0.0558,  0.2753],
         [-0.3959,  0.3549,  0.2270,  ...,  0.2509,  0.1121,  0.4615],
         [-0.4324,  0.3416,  0.2218,  ...,  0.2357,  0.1719,  0.5027],
         ...,
         [-0.5038,  0.4114,  0.3261,  ...,  0.1328,  0.2192,  0.5432],
         [-0.5051,  0.4052,  0.3402,  ...,  0.1590,  0.2244,  0.5482],
         [-0.4927,  0.4033,  0.3281,  ...,  0.1604,  0.1894,  0.5238]],

        [[-0.3173,  0.0634,  0.2716,  ...,  0.1602, -0.2058,  0.2507],
         [-0.6162,  0.0627,  0.1128,  ...,  0

- 94 is the number of sentences in sentences_train. It all fits in one batch since it is less than 128.
- 56 is the internal sequence length
- 384 is the length of the embedding 

In [55]:
outputs.last_hidden_state.shape

torch.Size([94, 56, 384])

In [56]:
pool_type

'avg'

When we are loading the data, we are showing where there is no padding with the attention mask>

In [57]:
batch_dict['attention_mask']

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')

In [58]:
batch_dict['attention_mask'].shape

torch.Size([94, 56])

In [59]:
last_hidden_states = outputs.last_hidden_state

In [60]:
attention_mask = batch_dict['attention_mask']

attention_mask[..., None] converts [94, 56] into [94, 56, 1]. It adds an extra dimension.

In [61]:
attention_mask[..., None].shape

torch.Size([94, 56, 1])

Bool converts this into True or Fale Values. ~ is going to invert these values

In [62]:
~attention_mask[..., None].bool()

tensor([[[False],
         [False],
         [False],
         ...,
         [ True],
         [ True],
         [ True]],

        [[False],
         [False],
         [False],
         ...,
         [ True],
         [ True],
         [ True]],

        [[False],
         [False],
         [False],
         ...,
         [ True],
         [ True],
         [ True]],

        ...,

        [[False],
         [False],
         [False],
         ...,
         [ True],
         [ True],
         [ True]],

        [[False],
         [False],
         [False],
         ...,
         [ True],
         [ True],
         [ True]],

        [[False],
         [False],
         [False],
         ...,
         [ True],
         [ True],
         [ True]]], device='cuda:0')

In [63]:
# This line is simply zeroing out anything where the attention mask is 0 (this is typically padding tokens)
last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)

In [64]:
last_hidden.shape

torch.Size([94, 56, 384])

In this process, I am averaging across the 56 internal dimensions that are produced by E5. 
I am doing this so that I can get one embedding (length 384) for each of the 94 sentences that I am training.

In [67]:
# sum along the internal sequence length. It is a sum of the 56 elements in dimension 1.
last_hidden_numerator = last_hidden.sum(dim=1)

In [68]:
last_hidden_numerator.shape

torch.Size([94, 384])

In [69]:
attention_mask.shape

torch.Size([94, 56])

Summing the attention mask is going to serve as the denominator. It is equivalent the the denominator when computing mean. 

In [71]:
attention_mask.sum(dim=1)[..., None].shape

torch.Size([94, 1])

In [72]:
# Summing the attention mask allows the average to happen

In [73]:
emb = last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

This now shows that the pooling operation ("avg") has been completed. 

In [74]:
emb.shape

torch.Size([94, 384])

In [75]:
# putting this all together results in the pool function 
def pool(last_hidden_states: Tensor,
         attention_mask: Tensor,
         pool_type: str) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)

    if pool_type == "avg":
        emb = last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]
    elif pool_type == "cls":
        emb = last_hidden[:, 0]
    else:
        raise ValueError(f"pool_type {pool_type} not supported")

    return emb

In [76]:
embeds = pool(outputs.last_hidden_state, batch_dict['attention_mask'], pool_type)

#### Execution of DenseEncoder

In [77]:
# cust/sedw/notes/nlp is the combindation of all of the above in the encode part of a custom torch.nn.Module 

In [78]:
model = de.DenseEncoder(
    model_name_or_path='intfloat/e5-small-v2'
)

2024-01-22 20:27:46,235 - MyAppLogger - INFO - Loading model intfloat/e5-small-v2...
2024-01-22 20:27:46,889 - MyAppLogger - INFO - GPU count: 1


In [79]:
# encode the sentences as embeddings using the e5-small-v2 model
X_train = np.asarray(model.encode(sentences_train, batch_size=batch_size))

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [80]:
# Each embedding contains 384 numbers
X_train[0].shape

(384,)

In [81]:
# X_train[0]

In [82]:
X_train.shape

(94, 384)

In [83]:
X_test = np.asarray(model.encode(sentences_test, batch_size=batch_size))

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


X_test represents a holdout of 20%

In [84]:
X_test.shape

(24, 384)

In [87]:
test_cache = X_test

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

##### Explanation of Logistic Regression

In [109]:
X_train[0]

array([-0.51499236,  0.14914134,  0.55764234,  0.08547238, -0.23854885,
        0.3037106 ,  0.3354031 , -0.36028975, -0.07882987,  0.10156981,
        0.0577091 , -0.26653755, -0.09946077,  0.21468602,  0.2277394 ,
       -0.06360664, -0.28164575,  0.7525943 , -0.4986084 ,  0.5038393 ,
        0.5938164 , -0.45927   , -0.00303126, -0.7904557 , -0.00307554,
        0.48803908,  0.431343  ,  0.54404616, -0.405484  , -1.0808046 ,
       -0.48698422, -0.12839562, -0.02057808, -0.23252465,  0.01595651,
       -0.44543445,  0.15081415, -0.0349281 ,  0.26383168,  0.14510569,
       -0.29242676, -0.5643637 ,  0.20347795, -0.81849635, -0.11122759,
       -0.16483097, -0.36363545, -0.410171  ,  0.44259813,  0.0502857 ,
       -0.500818  ,  0.03072342,  0.22073905,  0.3136644 ,  0.35880774,
       -0.06409074,  0.4261081 ,  0.05246903,  0.39038613,  0.28953096,
        0.2384326 ,  0.5947179 , -1.4262769 ,  1.2155329 ,  0.67613673,
        0.62881887, -0.2096899 , -0.04804755, -0.14080143,  0.11

multi_class is set to 'auto.' As seen in https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html, this means that 'ovr' (One-vs-Rest) approach is going to be used because this is a binary classification. 


Since this is binary classification, it seems that scikit learn still defaults to binary cross-entropy loss (aka log loss). Cross-entropy loss for single observation is:

$
Loss = -[y \log(p) + (1 - y) \log(1 - p)]$


and the average of the individual losses represents the loss function that we are going to use. This is :

$$
-\frac{1}{N} \sum_{i=1}^{N} \left[ y_i \log(p_i) + (1 - y_i) \log(1 - p_i) \right]
$$


In this formula:
- $N$ is the number of samples.
- $y_i$ is the actual label of the $i$-th sample.
- $p_i$ is the predicted probability of the $i$-th sample being in the positive class.

To illustrate how cross entropy works, lets take a look at a more simple example first. 

What if we only had 5 data points (instead of the 94)? What if those data points were:

$
\begin{align*}
1. (x_1, y_1) = (0.5, 0) \\
2. (x_2, y_2) = (1.5, 0) \\
3. (x_3, y_3) = (2.5, 1) \\
4. (x_4, y_4) = (3.5, 1) \\
5. (x_5, y_5) = (4.5, 1) \\
\end{align*}
$


If we started with a baseline of $b_0$ = 0 and $b_1$ = 1, then the logistic function would be $z = b_0 + b_1x_1$ or $z = 0 + 1 x 0.5. 

The predicted probability is $$
p = \frac{1}{1 + e^{-z}}
$$

In [114]:
# The predicted probability 
import math
p = 1 / (1 + math.exp(-0.5))

In [115]:
p

0.6224593312018546

Go back to the loss function above. Since y is 0 , $
Loss = -[y \log(p) + (1 - y) \log(1 - p)]
$ gets reduced to  $loss = -[log(1-p)]$

In [116]:
 1-p

0.3775406687981454

In [117]:
import numpy

In [118]:
np.log(1-p)

-0.9740769841801068

This means that the resulting loss for this observation is: + 0.974

Wrapping up this example, each of the other data points would get calculated in the same way to put together the loss function.

$$
-\frac{1}{N} \sum_{i=1}^{N} \left[ y_i \log(p_i) + (1 - y_i) \log(1 - p_i) \right]
$$

Now you need to calculate the gradient of the loss function with respect to each weight and bias. 
* You want to update the weights and biases in a manner that reduces the loss function.
* This is handled with an optimizer. The logistic regression in scikit-learn uses a sophisticated version of gradient decscent.
* 

##### Training the model

In [112]:
X_train.shape

(94, 384)

In [88]:
logistic_clf.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 20 concurrent workers.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =          385     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.93147D-01    |proj g|=  9.87033D-02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
  385     21     24      1     0     0   8.536D-05   1.031D-01
  F =  0.10308827952432373     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            


 This problem is unconstrained.


LogisticRegression(n_jobs=-1, random_state=42, verbose=1)

In [89]:
y_pred = logistic_clf.predict(X_test)

Accuracy is the proportion of correct preditions / the total number of predictions.

In [90]:
accuracy_score(y_test, y_pred)

1.0

$$ F1 = 2 \times \frac{{\text{precision} \times \text{recall}}}{{\text{precision} + \text{recall}}} $$


In [91]:
f1_score(y_test, y_pred, average="macro")

1.0

- Recall is the true positive rate.

- Average precision score is looking at 
$$
AP = \sum_n (R_n - R_{n-1}) P_n
$$
Where $P_n$ and $R_n$ are the precision and recall at the nth threshold.


In [92]:
average_precision_score(y_test, y_pred)

1.0

## Save Model

Use pickle in order to save the model.

In [93]:
# Remove any existing pickle files
!rm logistic_clf_model.pkl

In [94]:
import pickle

# Save the model to disk
with open('logistic_clf_model.pkl', 'wb') as model_file:
    pickle.dump(logistic_clf, model_file)


In [95]:
# confirm that the pickle file has been created
!ls | grep logis

logistic_clf_model.pkl


## Load and Classify

Load the model that was saved and use this as a sentence classifier.

In [96]:
import sys

In [97]:
import time

In [98]:
sys.path.append("/home/scott/whiteowlconsultinggroup/cust/sedw/maincode")
sys.path.append("/home/scott/whiteowlconsultinggroup/cust/sedw/maincode/api")
sys.path.append("/home/scott/whiteowlconsultinggroup/cust/sedw")
sys.path.append("/home/scott/whiteowlconsultinggroup/cust/sedw/maincode/api/wocg/classifiers/security")

In [99]:
import importlib

In [100]:
import denseencoder as de

In [101]:
importlib.reload(de)

<module 'denseencoder' from '/home/scott/whiteowlconsultinggroup/cust/sedw/maincode/api/wocg/classifiers/security/denseencoder.py'>

In [102]:
import pickle
import numpy as np

# Load the logistic regression model
with open('logistic_clf_model.pkl', 'rb') as model_file:
    loaded_logistic_clf = pickle.load(model_file)

# Load the DenseEncoder model
loaded_dense_encoder = de.DenseEncoder(model_name_or_path='intfloat/e5-small-v2')
loaded_dense_encoder.to_device('cpu')  # Move to CPU

def classify_sentence(sentence: str, loaded_model, loaded_encoder_model) -> int:
    """
    Classify the given sentence.
    1 if it's a document security question.
    0 if it's a window and door question.

    Parameters:
    sentence (str): The sentence to classify.
    model_path (str): The path to the saved logistic regression model.
    encoder_model (DenseEncoder): Preloaded DenseEncoder model to use for encoding the sentence.

    Returns:
    int: The class label.
    """

    # Use the pre-loaded models
    logistic_clf = loaded_model
    encoder_model = loaded_encoder_model

    # Encode the sentence
    encoded_sentence = np.asarray(encoder_model.encode([sentence]))

    # Predict the class
    prediction = logistic_clf.predict(encoded_sentence)

    return prediction[0]




2024-01-22 20:30:11,156 - MyAppLogger - INFO - Loading model intfloat/e5-small-v2...
2024-01-22 20:30:11,971 - MyAppLogger - INFO - GPU count: 1


In [103]:
sentence_to_classify = "What kind of warranties does SEDW offer on window installation?"

In [105]:
# Usage example
start_time = time.time()  # Record the start time
result = classify_sentence(sentence_to_classify, loaded_model=loaded_logistic_clf, loaded_encoder_model=loaded_dense_encoder)  # model is the existing DenseEncoder instance
end_time = time.time()  # Record the end time
elapsed_time = end_time - start_time  # Calculate elapsed time
print(f"Time taken to predict: {elapsed_time:.4f} seconds")
print(f"The sentence is a {'document security' if result == 1 else 'window and door'} question.")

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Time taken to predict: 0.1569 seconds
The sentence is a window and door question.


In [106]:
sentence_to_classify = "List all of the users in the system"

In [107]:
# Usage example
start_time = time.time()  # Record the start time
result = classify_sentence(sentence_to_classify, loaded_model=loaded_logistic_clf, loaded_encoder_model=loaded_dense_encoder)  # model is the existing DenseEncoder instance
end_time = time.time()  # Record the end time
elapsed_time = end_time - start_time  # Calculate elapsed time
print(f"Time taken to predict: {elapsed_time:.4f} seconds")
print(f"The sentence is a {'document security' if result == 1 else 'window and door'} question.")

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Time taken to predict: 0.1076 seconds
The sentence is a document security question.
